# Rotations Euler

Kevin J. Walchko, Phd

---

## Euler Rotations

The mathematical transformation between a body fixed frame and an inertial (non-moving) frame can be done with a rotation matrix. Euler developed a series of them, with the Aerospace Industry settling on the 3-2-1 sequence to go between a moving body and an inertial frame.

Here, the $R_{321} = R_{zyx} = R(\psi)R(\theta)R(\phi)$ is the standard. Thus a vector in body space is related to its inertial vector by $v^I = R_{321} v^b$. The sequence is:

![](dcm.png)

The simplest case occurs when the inertial and body frames align, then the matix is the identity:

![](identity.png)

## Calculating Euler Angles from Body Rates (Gyros)

Given the values $\begin{bmatrix} p & q & r \end{bmatrix}^T$ from a gyro, we want to be able to find the Euler angles ($\Phi =\begin{bmatrix} \phi & \theta & \psi \end{bmatrix}^T$). The relationship between the two are:

![](euler_rotation.png)

This is true when integrating fast and the angle are small values. You end up with the following equations:

$$
\begin{bmatrix}
p \\
q \\
r
\end{bmatrix} = 
\begin{bmatrix}
\dot \phi \\
0 \\
0
\end{bmatrix}^b + R(\phi) \begin{bmatrix}
0 \\
\dot \theta \\
0
\end{bmatrix}^{b'} + R(\phi) R(\theta) \begin{bmatrix}
0 \\
0 \\
\dot \psi
\end{bmatrix}^{b''} = \begin{bmatrix}
1 & 0      & -s\theta \\
0 & c\phi  & s\phi c\theta \\
0 & -s\phi & c\phi c\phi
\end{bmatrix} \begin{bmatrix}
\dot \phi \\
\dot \theta \\
\dot \psi
\end{bmatrix} = L^B_I \dot \Phi
$$

where $R$ is direct cosine matrix (DCM). Now $L^{-1} \neq L^T$ since this is not a symetric matrix. However, this can be rearranged to get:

$$
\dot \Phi =
\begin{bmatrix}
\dot \phi \\
\dot \theta \\
\dot \psi
\end{bmatrix} = 
\begin{bmatrix}
1 & s\phi t\theta & c\phi t\theta \\
0 & c\phi & -s\phi \\
0 & s\phi sec\theta & c\phi sec\theta
\end{bmatrix} \begin{bmatrix}
p \\
q \\
r
\end{bmatrix} = L^I_B \omega_B
$$

Now you could integrate this equation in order to update the Euler angles given gyro measurements ($\Phi \mathrel{{+}{=}} \omega_B \Delta t$). However, Euler angles have issues (i.e., gimbal lock) so typically engineers use quaternions instead to track the orientation of a rigid body. However, since quaternions are a 4D representation of 3D space, they are highly non-intuitive. Therefore, the quaternion is typically transformed back into Euler angles for human understanding only.

In [9]:
from math import sin, cos, tan, acos, pi
import numpy as np

# Derivative function
def xdot(x,u):
    x_dot = np.zeros(3)  # [0,0,0]
    #   phidot = p + (q*sin(phi) + r*cos(phi))*tan(theta)
    x_dot[0] = u[0] + (u[1]*sin(x[0]) + u[2]*cos(x[0]))*tan(x[1])
    #   thetadot = q*cos(phi) - r*sin(phi)
    x_dot[1] = u[1]*cos(x[0]) - u[2]*sin(x[0])
    #   psidot = (q*sin(phi) + r*cos(phi))*sec(theta)
    x_dot[2] = (u[1]*sin(x[0]) + u[2]*cos(x[0]))*acos(x[1])
    return x_dot

# Initial condition setup
eulerAngles = np.zeros(3)  # Assume start up with phi = theta = psi = 0 radians
deltaT = 0.01              # Sampling time (sec)

# loop
for i in range(10):
    omegaRead = [0.01,0.1,0.1] # gyros.getAngularVelocity()
    eulerAngles += xdot(eulerAngles,omegaRead) * deltaT
    
print(f"{np.rad2deg(eulerAngles)} deg")

[0.05987511 0.572693   0.89783636] deg


# References

- [Explanation of Euler Rotations](https://charlestytler.com/measuring-quadcopter-attitude-sensors/)